# Understanding Retrivals

- Suff Method
- MapReduce Method 

In [13]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS


In [14]:
#load openAI api key
os.environ['OPENAI_API_KEY'] = 'sk-proj-VZBi9kjb1DmxxDXHa0h2T3BlbkFJOPfgDcZ8ttJ9i1O3mH6S'

In [15]:
# Initialise LLM with required params
llm = OpenAI(temperature=0.9, max_tokens=500) 

In [16]:
loaders = UnstructuredURLLoader(
    urls = [
        "https://www.moneycontrol.com/news/business/stocks/accumulate-tata-motors-target-of-rs-1075-prabhudas-lilladher-12538061.html",
        "https://www.moneycontrol.com/news/business/stocks/buy-tata-motors-target-of-rs-1188-sharekhan-12411611.html",
        "https://www.moneycontrol.com/news/business/stocks/reduce-tata-motors-target-of-rs-901-icici-securities-12411521.html"
        ]
)
data = loaders.load() 
len(data)

3

In [17]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [18]:
# Create the embeddings of the chunks using openAIEmbeddings
embeddings = OpenAIEmbeddings()

# Pass the documents and embeddings inorder to create FAISS vector index
vectorindex_openai = FAISS.from_documents(docs, embeddings)

In [19]:
pkl = vectorindex_openai.serialize_to_bytes()

In [20]:
# Storing vector index create in local
file_path="vector_index_1.pkl"
with open(file_path, "wb") as f:
    pickle.dump(pkl, f)

In [21]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [22]:
vectorIndex = FAISS.deserialize_from_bytes(embeddings=OpenAIEmbeddings(), serialized=vectorIndex,allow_dangerous_deserialization=True)

In [23]:
chain = RetrievalQAWithSourcesChain.from_llm(llm=llm, retriever=vectorIndex.as_retriever())
chain

RetrievalQAWithSourcesChain(combine_documents_chain=MapReduceDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['context', 'question'], template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=OpenAI(client=<openai.resources.completions.Completions object at 0x000001FE149F7E90>, async_client=<openai.resources.completions.AsyncCompletions object at 0x000001FE7D6BED10>, temperature=0.9, max_tokens=500, openai_api_key='sk-proj-VZBi9kjb1DmxxDXHa0h2T3BlbkFJOPfgDcZ8ttJ9i1O3mH6S', openai_proxy='')), reduce_documents_chain=ReduceDocumentsChain(combine_documents_chain=StuffDocumentsChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'summaries'], template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t

In [24]:
query = "What is the least and highest suggested value for tata motors"

langchain.debug=True

chain({"question": query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is the least and highest suggested value for tata motors"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Story continues below Advertisement\n\nRemove Ad\n\nOutlook\n\nWe downgrade TTMT to REDUCE from Hold, post its ~40% rally in past three months, with an SoTP-based TP of INR 901 (implying ~13x/2.5x FY26E India/JLR EV/EBITDA).\n\nFor all recommendations report, click here\n\nRelated stories\n\nBuy Nazara Technologies; target of Rs 1080: ICICI Securities\n\nBuy City Union Bank; target of Rs 205: Asit C Mehta\n\nHold Nazara Technologies; target of Rs 878: Prabhudas Lilladher\n\nDisclaimer: The views and investment tips expressed by investm

{'answer': ' The least and highest suggested value for Tata Motors is INR 901 and INR 1188, respectively.\n',
 'sources': 'https://www.moneycontrol.com/news/business/stocks/reduce-tata-motors-target-of-rs-901-icici-securities-12411521.html, https://www.moneycontrol.com/news/business/stocks/buy-tata-motors-target-of-rs-1188-sharekhan-12411611.html, https://www.moneycontrol.com/news/business/stocks/accumulate-tata-motors-target-of-rs-1075-prabhudas-lilladher-12538061.html'}